# Exercise 4, answers

In [5]:
def f_constrained(x):
    return x[0]**2+x[1]**2+x[0]+2*x[1], [], [x[0]+x[1]-1]

In [4]:
def alpha(x,f):
    (_,ieq,eq) = f(x)
    return sum([min([0,ieq_j])**2 for ieq_j in ieq])\
+sum([eq_k**2 for eq_k in eq])

def penalized_function(x,f,r):
    return f(x)[0] + r*alpha(x,f)

#### Let us solve the penalized problem with penalty term growin in a loop

In [19]:
import numpy as np
from scipy.optimize import minimize
r = 0
x_old = np.array([float('inf')]*2)
x_new = [0,0]
while np.linalg.norm(x_new-x_old)>0.0001:
    res = minimize(lambda x:penalized_function(x,f_constrained,r),
               [0,0],method='Nelder-Mead')
    x_old = x_new
    x_new = np.array(res.x)
    r = r+1
print x_new, r

[ 0.74340989  0.24336301] 95


## Problem 2

In [20]:
def f_constrained_approx(x,epsilon):
    return x[0]**2+x[1]**2+x[0]+2*x[1], [x[0]+x[1]-1+epsilon,\
                                         epsilon-(x[0]+x[1]-1)], []

In [22]:
def beta(x,f):
    _,ieq,_ = f(x)
    try:
        value=sum([1/max([0,ieq_j]) for ieq_j in ieq])
    except ZeroDivisionError:
        value = float("inf")
    return value
def function_with_barrier(x,f,r):
    return f(x)[0]+r*beta(x,f)

In [38]:
import numpy as np
import ad
from scipy.optimize import minimize
r = 1.0
epsilon = 1.
x_old = np.array([float('inf')]*2)
x_new = [1,0]
while np.linalg.norm(x_new-x_old)>0.0001:
    g = lambda x: function_with_barrier(x,\
               lambda y: f_constrained_approx(y,epsilon),r)
    res = minimize(g,x_new,method='Newton-CG',jac=ad.gh(g)[0],\
                   hess=ad.gh(g)[1])
    x_old = x_new
    x_new = res.x
    r=r/2
    epsilon = epsilon/2
    print x_new, epsilon, r

[ 0.5578683  0.0578683] 0.5 0.5
[ 0.68545665  0.18545665] 0.25 0.25
[ 0.73158293  0.23158293] 0.125 0.125
[ 0.74519333  0.24519333] 0.0625 0.0625
[ 0.74878417  0.24878417] 0.03125 0.03125
[ 0.74969513  0.24969513] 0.015625 0.015625
[ 0.74992373  0.24992373] 0.0078125 0.0078125
[ 0.74998093  0.24998093] 0.00390625 0.00390625


## Problem 3

In [40]:
import numpy as np
def project_vector(A,vector):
    

In [42]:
import numpy as np
import ad
def projected_gradient_method(f,A,start,step,precision):
    f_old = float('Inf')
    x = np.array(start)
    steps = []
    f_new = f(x)
    while abs(f_old-f_new)>precision:
        f_old = f_new
        gradient = ad.gh(f)[0](x)
        grad_proj = project_vector(A,[-i for i in gradient])#The only changes to steepest..
        grad_proj = np.array(grad_proj.transpose())[0] #... descent are here!
        x = x+grad_proj*step
        f_new = f(x)
        steps.append(list(x))
    return x,f_new,steps

In [44]:
projected_gradient_method(lambda x:f_constrained(x)[0],[[1,1]],[1,0]\
                          ,.5,0.000001)

(array([ 0.75,  0.25]), 1.875, [[0.75, 0.25], [0.75, 0.25]])

## Problem 4

Need to show that there exists unique Lagrance multiplier vectors $\lambda^* = (\lambda^*_1,\ldots,\lambda_J^*)$ and $\mu^*=(\mu_1^*,\ldots,\mu_K^*)$ such that
$$
\begin{align}
&\nabla_xL(x,\lambda,\mu) = 0\\
&\mu_j^*\geq0,\text{ for all }j=1,\ldots,J\\
&\mu_j^*g_j(x)=0,\text{for all }j=1,\ldots,J,
\end{align}
$$
where $$L(x,\lambda,\mu) = f(x)- \sum_{k=1}^K\mu_kg_k(x) -\sum_{j=1}^J\lambda_jh_j(x)$$

Now,  $f(x) = x_1^2+x_2^2+x_1+2x_2$, $g(x) = 0$ and $h(x)=x_1+x_2-1$.

Thus, stability rule becomes $$
\left\{
\begin{align}
2x_1+1-\lambda = 0\\
2x_2+2-\lambda=0.
\end{align}
\right.
$$


We do not have a complementary rule, since we do not have inequality constraints!

In [52]:
def check_KKT_eqc(x,tol):
    l = 2*x[0]+1
    if abs(2*x[1]+2-l)<=tol:
        print abs(2*x[1]+2-l)
        return True
    return False

In [53]:
check_KKT_eqc([0.74998093,0.24998093],0.000001)

0.0


True